In [1]:
import time
import numpy as np
import pandas as pd
 
from bs4 import BeautifulSoup
import urllib.request as req

import requests
import os
import datetime
import re

In [3]:
import sys
sys.path.append("../")

In [4]:
from product_csv.create_df import game_df

In [5]:
# ディビジョンを選択してください:{v1_m, v2_m, v3_m, v1_w, v2_w}
division = 'v1_m'
# '年度を選択してください:{2019-20, 2018-19, 2017-18...}'
season = '2020-21'
# シーズンを選択してください:{regular, 2019-20(game1, game2, semi_final, final), 2014-15~2018-19(final6, final3, final),
# ~2013-14(semi_final, final)}
s_round = 'regular'
season += '_' + s_round
season

'2020-21_regular'

In [6]:
# 2020-21の更新
os.chdir('/Users/sasno/Desktop/MyPandas/vleague')

headers = {"User-Agent": "Mozilla/5.0"}
 
# seasons = get_seasons(division)
 
division = "v1_m"
# s_id = "283" # 2019-20
s_id = "301"
# division = "v1_w"
# s_id = "303"
# division = "v2_m"
# s_id = "299"
# division = "v2_w"
# s_id = "302"
# division = "v3_m"
# s_id = "300"
# division = "v3_w"
# s_id = "268" 
s_round = "2020-21_regular"
 
if not os.path.isdir(division):
    os.makedirs(division)
 
os.chdir(division)
print(os.getcwd())
 
 

game_all = None
 
# ページ数を取得
url = 'https://www.vleague.jp/round/list/{}'.format(s_id)
print(url)
request = req.Request(url, headers=headers)
response = req.urlopen(request)
parse_html = BeautifulSoup(response, 'html.parser')
hrefs = parse_html.find_all('a', href=re.compile("pg"))
 
# for i in hrefs:
#     print(i.text)
if len(hrefs) > 0:
    pages = int(hrefs[-2].text)
    print(pages)
 
    all_list = []
    for pg in range(1,pages+1):
        url = 'https://www.vleague.jp/round/list/{0}?pg={1}'.format(s_id,pg)
        print(url)
        game_df(url,headers,s_round,all_list)
else:
    all_list = []
    url = 'https://www.vleague.jp/round/list/{}'.format(s_id)
    print(url)
    game_df(url,headers,s_round,all_list)
try:
    game_all = pd.concat(all_list, ignore_index=True)
    print(game_all)
except:
    pass
all_dir = 'all/{}'.format(s_round)
if not os.path.isdir(all_dir):
    os.makedirs(all_dir)
 
print(os.getcwd())
try:
    daily_all = game_all[game_all['名前'] != 'チーム合計']
    print(daily_all)
 
    data = daily_all
    data.insert(1, '月', data['試合日'].str[5:7])
    data.set_index('試合日')
    monthly_all = data.groupby(['名前','月','チーム','背番号']).sum().reset_index()
    monthly_all['アタック決定率'] = ((monthly_all['アタック得点'] / monthly_all['アタック打数']) * 100).round(1)
    monthly_all['アタック効果率'] = (((monthly_all['アタック得点']-monthly_all['アタック失点']) / monthly_all['アタック打数']) * 100).round(1)
    monthly_all['バックアタック決定率'] = ((monthly_all['バックアタック得点'] / monthly_all['バックアタック打数']) * 100).round(1)
    monthly_all['アタックセット平均'] = (monthly_all['アタック得点'] / monthly_all['出場数']).round(2)
    monthly_all['ブロックセット平均'] = (monthly_all['ブロック得点'] / monthly_all['出場数']).round(2)
    monthly_all['サーブ効果率'] = ((monthly_all['サーブ得点'] * 100 + monthly_all['サーブ効果'] * 25 - monthly_all['サーブ失点'] * 25) / monthly_all['サーブ打数']).round(1)
    monthly_all['サーブレシーブ成功率'] = ((monthly_all['サーブレシーブ成功・優'] * 100 + monthly_all['サーブレシーブ成功・良'] * 50) / monthly_all['受数']).round(1)
    print(monthly_all)
 
    yearly_all = data.groupby(['名前','チーム','背番号']).sum().reset_index()
    yearly_all['アタック決定率'] = ((yearly_all['アタック得点'] / yearly_all['アタック打数']) * 100).round(1)
    yearly_all['アタック効果率'] = (((yearly_all['アタック得点']-yearly_all['アタック失点']) / yearly_all['アタック打数']) * 100).round(1)
    yearly_all['バックアタック決定率'] = ((yearly_all['バックアタック得点'] / yearly_all['バックアタック打数']) * 100).round(1)
    yearly_all['アタックセット平均'] = (yearly_all['アタック得点'] / yearly_all['出場数']).round(2)
    yearly_all['ブロックセット平均'] = (yearly_all['ブロック得点'] / yearly_all['出場数']).round(2)
    yearly_all['サーブ効果率'] = ((yearly_all['サーブ得点'] * 100 + yearly_all['サーブ効果'] * 25 - yearly_all['サーブ失点'] * 25) / yearly_all['サーブ打数']).round(1)
    yearly_all['サーブレシーブ成功率'] = ((yearly_all['サーブレシーブ成功・優'] * 100 + yearly_all['サーブレシーブ成功・良'] * 50) / yearly_all['受数']).round(1)
    print(yearly_all)
 
    game_all.to_csv('all/{}/game_all.csv'.format(s_round), index=False, encoding='cp932')
    daily_all.to_csv('all/{}/daily_all.csv'.format(s_round), index=False, encoding='cp932')
    monthly_all.to_csv('all/{}/monthly_all.csv'.format(s_round), index=False, encoding='cp932')
    yearly_all.to_csv('all/{}/yearly_all.csv'.format(s_round), index=False, encoding='cp932')
    print('csv作成成功')
except:
    pass

c:\Users\sasno\Desktop\MyPandas\vleague\v1_m
https://www.vleague.jp/round/list/301
9
https://www.vleague.jp/round/list/301?pg=1
['https://www.vleague.jp//form/b/25724', 'https://www.vleague.jp//form/b/25722', 'https://www.vleague.jp//form/b/25716', 'https://www.vleague.jp//form/b/25718', 'https://www.vleague.jp//form/b/25720', 'https://www.vleague.jp//form/b/25717', 'https://www.vleague.jp//form/b/25721', 'https://www.vleague.jp//form/b/25725', 'https://www.vleague.jp//form/b/25723', 'https://www.vleague.jp//form/b/25719', 'https://www.vleague.jp//form/b/25726', 'https://www.vleague.jp//form/b/25732', 'https://www.vleague.jp//form/b/25728', 'https://www.vleague.jp//form/b/25734', 'https://www.vleague.jp//form/b/25727', 'https://www.vleague.jp//form/b/25729', 'https://www.vleague.jp//form/b/25733', 'https://www.vleague.jp//form/b/25735', 'https://www.vleague.jp//form/b/25730', 'https://www.vleague.jp//form/b/25731']
https://www.vleague.jp/round/list/301?pg=2
['https://www.vleague.jp//fo

In [7]:
import tweepy
import os
import glob
import time
import numpy as np
import pandas as pd
import re
 
# key
consumer_key = 'D4OAzhqT7NNeeCa9R7h0Hxzx3'
consumer_secret = 'Py8P1k5jxdPEvNOb64WUXh8b10sfDL5qr60krVLNOOLEXauZti'
access_token = '1309887734555668481-NNvdk0MCxK4WwKZCKZrOqflU9bwPfG'
access_secret = 'CmkMLM4eiOq4MLP3TE6FfUobThOElR5OAfuVSv3GRnXrJ'
 
# Twitterオブジェクト
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [8]:
daily_all = pd.read_csv('all/{}/daily_all.csv'.format(s_round),encoding='cp932')
# daily_all
day = '2020-10-31'
today_data = daily_all[daily_all['試合日']==day]
# today_data.head()
stats_list = ['total', 'at_attack','attack_k', 'b_attack_k', 'block', 'surve_k', 'surve_eff', 'cut']
# stats = np.random.choice(stats_list)
for stats in stats_list:
    print(stats)
    if stats == 'total':
        reg_judge = today_data['出場数']>=1
        column = '総得点'
        pattern = ['出場数1セット', 2]
    elif stats == 'at_attack':
        reg_judge = today_data['出場数']>=1
        column ='アタック打数'
        pattern = ['出場数1セット', 0]
    elif stats == 'attack_k':
        reg_judge = today_data['アタック打数']>=10
        column = 'アタック決定率'
        pattern = ['打数10本', 1]
    elif stats == 'b_attack_k':
        reg_judge = today_data['バックアタック打数']>=5
        column = 'バックアタック決定率'
        pattern = ['打数5本', 1]
    elif stats == 'block':
        reg_judge = today_data['出場数']>=2
        column = 'ブロックセット平均'
        pattern = ['出場数2セット', 0]
    elif stats == 'surve_k':
        reg_judge = today_data['サーブ打数']>=5
        column = 'サーブ得点'
        pattern = ['サーブ打数5本', 2]
    elif stats == 'surve_eff':
        reg_judge = today_data['サーブ打数']>=5
        column = 'サーブ効果率'
        pattern = ['サーブ打数5本', 1]
    else:
        reg_judge = today_data['受数']>=5
        column = 'サーブレシーブ成功率'
        pattern = ['受数5本', 1]
    print(column)

    for i in range(3):
        stats_df = today_data[reg_judge][['名前', 'チーム', '背番号', '出場数', column]]
        stats_df = stats_df.sort_values(column, ascending=False).head()
        print(stats_df)
        if len(stats_df) >= 3:
            break
    # try:
    # stats_df = stats_df.to_dict()
    print(stats_df)
    if division == 'v1_m':
        division_j = 'V1男子'
    elif division == 'v1_w':
        division_j = 'V1女子'
    elif division == 'v2_m':
        division_j = 'V2男子'
    elif division == 'v2_w':
        division_j = 'V2女子'
    elif division == 'v3_m':
        division_j = 'V3男子'
    else:
        division_j = 'V3女子'
    
    p = re.compile('[\u0000-\u007F]+')
    messages = '本日({})の成績\n'.format(day)
    messages += 'シーズン: {}\n'.format(season)
    messages += '【{0}】ランキング({1}以上)\n'.format(column, pattern[0])
    messages += 'ディビジョン: {}\n'.format(division_j)
    print(len(stats_df))
    for i in range(len(stats_df)):
        if pattern[1]==1:
            message = '{0}位 #{1} ({2}, {3}%)\n'.format(i+1, stats_df['名前'].iloc[i], stats_df['チーム'].iloc[i], stats_df[column].iloc[i])
        elif pattern[1]==2:
            message = '{0}位 #{1} ({2}, {3}点)\n'.format(i+1, stats_df['名前'].iloc[i], stats_df['チーム'].iloc[i], stats_df[column].iloc[i])
        else:
            message = '{0}位 #{1} ({2}, {3}本)\n'.format(i+1, stats_df['名前'].iloc[i], stats_df['チーム'].iloc[i], stats_df[column].iloc[i])
        m_len = messages + message
        m_len = messages + message
        e_len = len(''.join(p.findall(m_len)))
        tweet_len = len(m_len) - (e_len/2)
        # print(tweet_len)
        if tweet_len <= 140:
            messages += message
        else:
            break
    print(messages)
    # api.update_status(messages)
    # except:
    #     pass

total
総得点
              名前          チーム  背番号  出場数  総得点
586     クレクバルトシュ   ウルフドッグス名古屋    3    3   24
684   パダルクリスティアン       東レアローズ    4    3   24
634  ムセルスキードミトリー   サントリーサンバーズ   13    3   23
567   プレモビッチピーター         FC東京   20    3   22
653         清水邦広  パナソニックパンサーズ    1    4   21
              名前          チーム  背番号  出場数  総得点
586     クレクバルトシュ   ウルフドッグス名古屋    3    3   24
684   パダルクリスティアン       東レアローズ    4    3   24
634  ムセルスキードミトリー   サントリーサンバーズ   13    3   23
567   プレモビッチピーター         FC東京   20    3   22
653         清水邦広  パナソニックパンサーズ    1    4   21
5
本日(2020-10-31)の成績
シーズン: 2020-21_regular
【総得点】ランキング(出場数1セット以上)
ディビジョン: V1男子
1位 #クレクバルトシュ (ウルフドッグス名古屋, 24点)
2位 #パダルクリスティアン (東レアローズ, 24点)
3位 #ムセルスキードミトリー (サントリーサンバーズ, 23点)

at_attack
アタック打数
              名前           チーム  背番号  出場数  アタック打数
648         袴谷亮介  ジェイテクトSTINGS   11    4      45
684   パダルクリスティアン        東レアローズ    4    3      45
586     クレクバルトシュ    ウルフドッグス名古屋    3    3      42
577  ストックトンカーティス  大分三好ヴァイセアドラー   11    3      38
567   プレモビッチピーター

In [7]:
daily_all = pd.read_csv('all/2020-21_regular/daily_all.csv',encoding='cp932')
daily_all.sort_values('総得点', ascending=False).head()

,試合日,月,チーム,相手チーム,背番号,リベロ,名前,出場数,1,2,...,サーブ打数,サーブ得点,サーブ失点,サーブ効果,サーブ効果率,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,サーブレシーブ成功率,総得点
150,2020-10-18,10,日立リヴァーレ,トヨタ車体クインシーズ,17,NaN,オクム大庭冬美ハウィ,5,■,■,...,10,0,0,3,7.5,0,0,0,-,40
189,2020-10-18,10,JTマーヴェラス,久光スプリングス,10,NaN,ドルーズアンドレア,4,■,■,...,8,1,1,4,21.9,0,0,0,-,27
267,2020-10-18,10,KUROBEアクアフェアリーズ,埼玉上尾メディックス,9,NaN,舛田紗淑,4,■,■,...,21,3,1,7,21.4,31,11,12,54.8,25
246,2020-10-18,10,NECレッドロケッツ,岡山シーガルズ,12,NaN,山内美咲,4,□,■,...,16,3,0,2,21.9,27,16,5,68.5,25
312,2020-10-24,10,埼玉上尾メディックス,ヴィクトリーナ姫路,9,NaN,内瀬戸真実,5,■,■,...,17,0,0,5,7.4,31,10,7,43.5,23


In [9]:
yearly_all = pd.read_csv('all/{}/yearly_all.csv'.format(s_round),encoding='cp932')
n = 5
stats_list = ['total', 'at_attack','attack_k', 'b_attack_k', 'block', 'surve_k', 'surve_eff', 'cut']
# stats = np.random.choice(stats_list)
for stats in stats_list:
    print(stats)
    if stats == 'total':
        reg_judge = yearly_all['出場数']>=n
        column = '総得点'
        pattern = ['出場数{}セット'.format(n), 2]
    elif stats == 'at_attack':
        reg_judge = yearly_all['出場数']>=n
        column ='アタック打数'
        pattern = ['出場数{}セット'.format(n), 0]
    elif stats == 'attack_k':
        reg_judge = yearly_all['アタック打数']>=n*10
        column = 'アタック決定率'
        pattern = ['打数{}本'.format(n*10), 1]
    elif stats == 'b_attack_k':
        reg_judge = yearly_all['バックアタック打数']>=n*4
        column = 'バックアタック決定率'
        pattern = ['打数{}本'.format(n*4), 1]
    elif stats == 'block':
        reg_judge = yearly_all['出場数']>=n
        column = 'ブロックセット平均'
        pattern = ['出場数{}セット'.format(n), 0]
    elif stats == 'surve_k':
        reg_judge = yearly_all['サーブ打数']>=n*5
        column = 'サーブ得点'
        pattern = ['サーブ打数{}本'.format(n*5), 2]
    elif stats == 'surve_eff':
        reg_judge = yearly_all['サーブ打数']>=n*5
        column = 'サーブ効果率'
        pattern = ['サーブ打数{}本'.format(n*5), 1]
    else:
        reg_judge = yearly_all['受数']>=n*10
        column = 'サーブレシーブ成功率'
        pattern = ['受数{}本'.format(n*10), 1]
    print(column)

    for i in range(3):
        stats_df = yearly_all[reg_judge][['名前', 'チーム', '背番号', '出場数', column]]
        stats_df = stats_df.sort_values(column, ascending=False).head()
        print(stats_df)
        if len(stats_df) >= 3:
            break
    # try:
    # stats_df = stats_df.to_dict()
    print(stats_df)
    p = re.compile('[\u0000-\u007F]+')
    messages = '今シーズン({})通算成績\n'.format(season)
    messages += '【{0}】ランキング({1}以上)\n'.format(column, pattern[0])
    messages += 'ディビジョン: {}\n'.format(division)
    print(len(stats_df))
    for i in range(len(stats_df)):
        if pattern[1]==1:
            message = '{0}位 #{1} ({2}, {3}%)\n'.format(i+1, stats_df['名前'].iloc[i], stats_df['チーム'].iloc[i], stats_df[column].iloc[i])
        elif pattern[1]==2:
            message = '{0}位 #{1} ({2}, {3}点)\n'.format(i+1, stats_df['名前'].iloc[i], stats_df['チーム'].iloc[i], stats_df[column].iloc[i])
        else:
            message = '{0}位 #{1} ({2}, {3}本)\n'.format(i+1, stats_df['名前'].iloc[i], stats_df['チーム'].iloc[i], stats_df[column].iloc[i])
        m_len = messages + message
        m_len = messages + message
        e_len = len(''.join(p.findall(m_len)))
        tweet_len = len(m_len) - (e_len/2)
        # print(tweet_len)
        if tweet_len <= 140:
            messages += message
        else:
            break
    print(messages)
    api.update_status(messages)
    # except:
    #     pass

total
総得点
           名前           チーム  背番号  出場数  総得点
131      西田有志  ジェイテクトSTINGS   14   17  136
2    クレクバルトシュ    ウルフドッグス名古屋    3   18  129
107      清水邦広   パナソニックパンサーズ    1   19   92
87       柳田将洋    サントリーサンバーズ    8   20   92
3     ジョンウェント       堺ブレイザーズ   24   19   82
           名前           チーム  背番号  出場数  総得点
131      西田有志  ジェイテクトSTINGS   14   17  136
2    クレクバルトシュ    ウルフドッグス名古屋    3   18  129
107      清水邦広   パナソニックパンサーズ    1   19   92
87       柳田将洋    サントリーサンバーズ    8   20   92
3     ジョンウェント       堺ブレイザーズ   24   19   82
5
今シーズン(2020-21_regular)通算成績
【総得点】ランキング(出場数5セット以上)
ディビジョン: v1_m
1位 #西田有志 (ジェイテクトSTINGS, 136点)
2位 #クレクバルトシュ (ウルフドッグス名古屋, 129点)
3位 #清水邦広 (パナソニックパンサーズ, 92点)
4位 #柳田将洋 (サントリーサンバーズ, 92点)

at_attack
アタック打数
           名前           チーム  背番号  出場数  アタック打数
2    クレクバルトシュ    ウルフドッグス名古屋    3   18     220
131      西田有志  ジェイテクトSTINGS   14   17     202
3     ジョンウェント       堺ブレイザーズ   24   19     163
87       柳田将洋    サントリーサンバーズ    8   20     155
101      池田幸太    VC長野トライデンツ    9   18     152